# Erlang Fixed-Point Approximation
Consider the 4-node ring network shown below.

![jupyter](./images/efpa.png)

Each of the four links has k = 20 circuits.

The traffic demands are as follows.

On Route: 4 → 1 → 2, the offered traffic is A(4, 1, 2) = 10 erlangs.

On Route: 4 → 1, the offered traffic is A(4, 1) = 11 erlangs.

On Route: 1 → 2, the offered traffic is A(1, 2) = 9 erlangs.

On Route: 2 → 3 → 4, the offered traffic is A(2, 3, 4) = 9 erlangs.

On Route: 2 → 3, the offered traffic is A(2, 3) = 12 erlangs.

On Route: 3 → 4, the offered traffic is A(3, 4) = 11 erlangs.

Find the blocking probability on routes 4 → 1 → 2, 4 → 1, 1 → 2, 2 → 3, 3 → 4, and 2 → 3 → 4.

## Erlang B Recursion

In [1]:
import numpy as np

In [2]:
def erlang_b_recursion(k, A):
    E = np.zeros(k + 1)
    E[0] = 1
    for m in range(1, k + 1):
        E[m] = A * E[m - 1] / (m + A * E[m - 1])
    return E[k]

## Traffic Demands of the Network
A and B are the Python dictionary (which has the same function as HashMap in Java).

The key-value pairs in dictionary A are the route and its corresponding offered traffic.

The key-value pairs in dictionary B are the route and its corresponding blocking probability.

In [3]:
# Number of circuits on each link
k = 20
# Offered traffic on routes (4,1,2), (4,1), (1,2), (2,3,4), (2,3), and (3,4)
A = {'412': 10, '41': 11, '12': 9, '234': 9, '23': 12, '34': 11}
# Blocking Probability on routes
B = {'412': 0, '41': 0, '12': 0, '234': 0, '23': 0, '34': 0}
loop_time = 10

## Blocking Probability for Route (4,1), (1,2), and (4,1,2)

In [4]:
# Total traffic offered to link (4,1) from traffic that flow on route (4,1)
a41 = A['41']
# Total traffic offered to link (1,2) from traffic that flow on route (1,2)
a12 = A['12']

# a412['41']: Total traffic offered to link (4,1) from traffic that flow on route (4,1,2)
# a412['12']: Total traffic offered to link (1,2) from traffic that flow on route (4,1,2)
a412 = {'41': 0, '12': 0}

for i in range(loop_time):
    a412['41'] = A['412'] * (1 - B['12'])
    a412['12'] = A['412'] * (1 - B['41'])
    B['41'] = erlang_b_recursion(k, A['41'] + a412['41'])
    B['12'] = erlang_b_recursion(k, A['12'] + a412['12'])

B['412'] = 1 - (1 - B['41']) * (1 - B['12'])
print("Blocking Probability for route (4, 1):", B['41'])
print("Blocking Probability for route (1, 2):", B['12'])
print("Blocking Probability for route (4, 1, 2):", B['412'])

Blocking Probability for route (4, 1): 0.16016011775854586
Blocking Probability for route (1, 2): 0.09497676464917473
Blocking Probability for route (4, 1, 2): 0.23992539259718304


## Blocking Probability for Route (2,3), (3,4), and (2,3,4)

In [5]:
# Total traffic offered to link (2,3) from traffic that flow on route (2,3)
a23 = A['23']
# Total traffic offered to link (3,4) from traffic that flow on route (3,4)
a34 = A['34']

# a234['23']: Total traffic offered to link (2,3) from traffic that flow on route (2,3,4)
# a234['34']: Total traffic offered to link (3,4) from traffic that flow on route (2,3,4)
a234 = {'23': 0, '34': 0}

for i in range(loop_time):
    a234['23'] = A['234'] * (1 - B['34'])
    a234['34'] = A['234'] * (1 - B['23'])
    B['23'] = erlang_b_recursion(k, A['23'] + a234['23'])
    B['34'] = erlang_b_recursion(k, A['34'] + a234['34'])

B['234'] = 1 - (1 - B['23']) * (1 - B['34'])
print("Blocking Probability for route (2, 3):", B['23'])
print("Blocking Probability for route (3, 4):", B['34'])
print("Blocking Probability for route (2, 3, 4):", B['234'])

Blocking Probability for route (2, 3): 0.15602379715292486
Blocking Probability for route (3, 4): 0.12373639589713038
Blocking Probability for route (2, 3, 4): 0.2604543707161673
